In [ ]:
!pip install torch torchvision numpy matplotlib scikit-learn tqdm

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/dataset_lukisan.zip'

extract_path = '/content/temp_dataset'

if not os.path.exists(extract_path):
    print(f"Sedang mengekstrak {zip_path} ke {extract_path}...")
    !unzip -q "$zip_path" -d "$extract_path"
    print("Ekstraksi selesai!")
else:
    print("Folder dataset sudah ada. Melewati proses ekstraksi.")

DATA_DIR = extract_path

TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VAL_DIR = os.path.join(DATA_DIR, 'val')

print(f"Isi folder DATA_DIR ({DATA_DIR}):")
print(os.listdir(DATA_DIR))

In [ ]:
class FFTTransform:
    def __call__(self, img):
        #Grayscale
        img_gray = img.convert('L')
        img_array = np.array(img_gray)

        #Fast Fourier Transform
        f = np.fft.fft2(img_array)
        fshift = np.fft.fftshift(f) #frekuensi rendah ke tengah

        #Hitung Magnitude Spectrum
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)

        #Normalisasi ke range 0-255
        magnitude_spectrum = np.nan_to_num(magnitude_spectrum)
        ms_min = np.min(magnitude_spectrum)
        ms_max = np.max(magnitude_spectrum)

        # Scaling min-max
        if ms_max - ms_min > 0:
            img_fft = 255 * (magnitude_spectrum - ms_min) / (ms_max - ms_min)
        else:
            img_fft = np.zeros_like(magnitude_spectrum)

        img_fft = img_fft.astype(np.uint8)

        #Konversi PIL Image
        img_fft_pil = Image.fromarray(img_fft).convert("RGB")

        return img_fft_pil

def visualize_fft_sample(image_path):
    img = Image.open(image_path)
    transformer = FFTTransform()
    fft_img = transformer(img)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[0].set_title("Original Image")
    ax[1].imshow(fft_img)
    ax[1].set_title("FFT Spectrum Input")
    plt.show()

visualize_fft_sample('/content/drive/MyDrive/dataset_lukisan/train/ai/sample.jpg')

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        FFTTransform(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        FFTTransform(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': torchvision.datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'val': torchvision.datasets.ImageFolder(VAL_DIR, data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(f"Classes: {class_names}")
print(f"Training samples: {dataset_sizes['train']}")
print(f"Validation samples: {dataset_sizes['val']}")

In [ ]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Freeze sebagian layer awal (opsional, tapi disarankan agar training lebih cepat)
# Jika dataset besar (22k), kita bisa unfreeze semua atau freeze sedikit saja.
# Di sini kita biarkan unfreezed agar model belajar fitur frekuensi dari awal.
# for param in model.features.parameters():
#     param.requires_grad = False

# Ganti Classifier Head terakhir
# EfficientNet-B0 classifier input features biasanya 1280
num_ftrs = model.classifier[1].in_features

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(num_ftrs, 2)
)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=10):
    best_acc = 0.0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            #Iterasi data
            for inputs, labels in tqdm(dataloaders[phase], desc=phase):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                #Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    #Backward + Optimize
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.item())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.item())

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), 'best_model_efficientnet_fft.pth')

    print(f'Best Val Acc: {best_acc:.4f}')
    return model, history

model_trained, history = train_model(model, criterion, optimizer, num_epochs=30)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

from google.colab import files
files.download('bestModelV1.pth')